In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("housing.csv")

In [3]:
data.sample(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
11109,-117.90,33.85,32.0,1605.0,314.0,986.0,306.0,3.3375,186200.0,<1H OCEAN
1547,-121.95,37.74,19.0,5721.0,837.0,2653.0,813.0,6.2631,266000.0,<1H OCEAN
6694,-118.09,34.14,40.0,3092.0,549.0,1457.0,536.0,5.3377,373800.0,INLAND
17275,-119.69,34.38,39.0,1383.0,459.0,677.0,362.0,2.2500,281300.0,NEAR OCEAN
12841,-121.46,38.66,3.0,3438.0,603.0,1602.0,554.0,3.9914,120500.0,INLAND


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [5]:
data.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [6]:
X=data.drop(columns='median_house_value')

In [7]:
X.sample(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
13103,-121.34,38.44,14.0,3205.0,465.0,1439.0,456.0,5.7452,INLAND
3608,-118.46,34.23,16.0,6338.0,1768.0,4718.0,1632.0,3.0187,<1H OCEAN
4693,-118.37,34.07,52.0,2203.0,437.0,899.0,384.0,4.2500,<1H OCEAN
2774,-115.80,33.26,2.0,96.0,18.0,30.0,16.0,5.3374,INLAND
12747,-121.37,38.63,37.0,494.0,86.0,253.0,99.0,4.8194,INLAND


In [8]:
Y=data['median_house_value']

In [9]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer

In [10]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [11]:
OHE=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
X_train_OHE=OHE.fit_transform(X_train)
X_test_OHE=OHE.transform(X_test)


In [12]:
Impute=SimpleImputer(strategy='median')
X_train_impute=Impute.fit_transform(X_train_OHE)
X_test_impute=Impute.transform(X_test_OHE)

In [13]:
Scaler=StandardScaler()
X_train_scaled=Scaler.fit_transform(X_train_impute)
X_test_scaled=Scaler.transform(X_test_impute)

In [14]:
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',     # feature randomness
    bootstrap=True,          # REQUIRED for OOB
    oob_score=True,          # enable OOB score
    random_state=42,
    n_jobs=-1
)


In [16]:
rf.fit(X_train_scaled,Y_train)

RandomForestRegressor(max_features='sqrt', n_jobs=-1, oob_score=True,
                      random_state=42)

In [18]:
Y_pred=rf.predict(X_test_scaled)

In [20]:
r2=r2_score(Y_test,Y_pred)
MSE=mean_squared_error(Y_test,Y_pred)
MAE=mean_absolute_error(Y_test,Y_pred)

In [21]:
print("r2 score is",r2)
print("MAE is",MAE)
print("MSE is",MSE)

r2 score is 0.49855814953902267
MAE is 57424.16786983204
MSE is 6581625404.664343


In [22]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}


In [24]:
grid=GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    verbose=1,
    n_jobs=-1
)

In [25]:
grid.fit(X_train_scaled,Y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


MemoryError: Unable to allocate 2.04 GiB for an array with shape (11559, 23661) and data type float64